# 注意：
此文件已在colab完成训练生成grade.h模型文件和对应的参数文件parameter.kpl

# 配置colab环境

In [ ]:
#colab中运行jupyter文件的步骤：
# 1.挂载云盘
from google.colab import drive
drive.mount('/content/gdrive')

# 2.安装需要的软件
!pip3 install transformers
!pip3 install pytorch-crf

import os
def get_root_dir():
    if os.path.exists('/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade'):
        return '/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade/'
    else:
        return './' #在本地

# 3.调用系统命令，切换到对应工程路径，相当于cd，但是直接!cd是不行的
print("path:",get_root_dir())
os.chdir(get_root_dir())

# 4.再次确认路径
!pwd
!ls

Mounted at /content/gdrive
     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 61 kB 610 kB/s 
     |████████████████████████████████| 895 kB 69.8 MB/s 
     |████████████████████████████████| 3.3 MB 61.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
path: /content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade/
/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade
grade.h5       意图识别推理及评测.ipynb		  打分算法-基于文本分类.ipynb
parameter.pkl  意图识别（文本分类）-最新版.ipynb  打分算法-推理.ipynb


In [ ]:
import os
import time
import random
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
from operator import itemgetter
from collections import defaultdict
import torch
import jieba

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.set_device(0)
else:
    device = torch.device('cpu')
print(device)
# 准备好模型的参数
parameter = {
    'epoch':100,
    'batch_size':300,
    'embedding_dim':300,
    'hidden_size':128,
    'num_layers':2, 
    'dropout':0.1,
    'cuda':torch.device('cuda'),
    #'cuda':torch.device('cpu'),
    'lr':0.001,
    'max_len':50,
}

def build_dataSet(parameter):
    data_src = pd.read_csv('../../dataSet/data_src.csv')
    data_src = data_src[data_src['关系'] == 'question2answer']
    q,a = list(data_src.实体1),list(data_src.实体2)
    word2id = defaultdict(int)
    word2id['<PAD>'] = 0
    word2id['<UNK>'] = 0
    qa_list = {}
    for ind in range(len(q)):
        q_cut = list(q[ind])
        a_cut = list(a[ind])
        if q[ind] not in qa_list:
            qa_list[q[ind]] = [q_cut,a_cut]
        else:
            qa_list[q[ind]] += [a_cut]
        for i in q_cut:
            word2id[i] += 1
        for i in a_cut:
            word2id[i] += 1
    qa_list = list(qa_list.values())
    parameter['qa_list'] = qa_list
    parameter['word2id'] = dict(zip(word2id.keys(),range(len(word2id))))
    parameter['id2word'] = dict(zip(range(len(word2id)),word2id.keys()))
    parameter['word_size'] = len(word2id)
    
def sample(n,parameter,neg_sample_num):
    neg_sample = []
    q_size = len(parameter['qa_list'])
    while 1:
        sample_id = random.randint(0,q_size-1)
        if sample_id == n:
            continue
        neg_sample_answer = parameter['qa_list'][sample_id]
        a_id = random.randint(1,len(neg_sample_answer)-1)
        neg_sample.append(neg_sample_answer[a_id])
        if len(neg_sample) >= neg_sample_num:
            return neg_sample
        
def list2torch(a):
    return torch.from_numpy(np.array(a)).long().to(parameter['cuda'])
    
def batch_yield(parameter,shuffle = True):
    for train_epoch in range(parameter['epoch']):
        qa_list = parameter['qa_list']
        data = []
        for ind,i in enumerate(qa_list):
            q = i[0]
            p_a = i[1:]
            n_a = sample(ind,parameter,len(p_a))
            q = [q] * len(p_a)
            data += list(zip(q,p_a,n_a))
        if shuffle:
            random.shuffle(data)
        batch_q,batch_a,batch_n = [],[],[]
        seq_len_q,seq_len_a,seq_len_n = 0,0,0
        for (q,a,n) in tqdm(data):
            q = itemgetter(*q)(parameter['word2id'])
            a = itemgetter(*a)(parameter['word2id'])
            n = itemgetter(*n)(parameter['word2id'])
            q = list(q) if type(q) == type(()) else [q,0]
            a = list(a) if type(a) == type(()) else [a,0]
            n = list(n) if type(n) == type(()) else [n,0]
            q = q[:parameter['max_len']]
            a = a[:parameter['max_len']]
            n = n[:parameter['max_len']]
            if len(q) > seq_len_q:
                seq_len_q = len(q)
            if len(a) > seq_len_a:
                seq_len_a = len(a)
            if len(n) > seq_len_n:
                seq_len_n = len(n)
            batch_q.append(q)
            batch_a.append(a)
            batch_n.append(n)
            if len(batch_q) >= parameter['batch_size']:
                batch_q = [i+[0]*(seq_len_q-len(i)) for i in batch_q]
                batch_a = [i+[0]*(seq_len_a-len(i)) for i in batch_a]
                batch_n = [i+[0]*(seq_len_n-len(i)) for i in batch_n]
                yield list2torch(batch_q),list2torch(batch_a),list2torch(batch_n),None,False
                batch_q,batch_a,batch_n = [],[],[]
                seq_len_q,seq_len_a,seq_len_n = 0,0,0
        batch_q = [i+[0]*(seq_len_q-len(i)) for i in batch_q]
        batch_a = [i+[0]*(seq_len_a-len(i)) for i in batch_a]
        batch_n = [i+[0]*(seq_len_n-len(i)) for i in batch_n]
        yield list2torch(batch_q),list2torch(batch_a),list2torch(batch_n),train_epoch,False
        batch_q,batch_a,batch_n = [],[],[]
        seq_len_q,seq_len_a,seq_len_n = 0,0,0
    yield None,None,None,None,True
if not os.path.exists('parameter.pkl'):
  print('parameter.pkl not exist!')
  build_dataSet(parameter)
  pk.dump(parameter,open('parameter.pkl','wb'))
else:
  print('parameter.pkl does exist!')

cuda:0
parameter.pkl not exist!


In [ ]:
#若不生成parameter.pkl这里执行会报错
train_yield = batch_yield(parameter)
test_q,test_a,test_n,_,_ = next(train_yield)
test_q,test_a,test_n

  0%|          | 0/1390 [00:00<?, ?it/s]

(tensor([[ 65, 349, 282,  ...,   0,   0,   0],
         [121, 122,  65,  ...,   0,   0,   0],
         [101,  99,  29,  ...,   0,   0,   0],
         ...,
         [170, 247, 366,  ...,   0,   0,   0],
         [287, 285, 337,  ...,   0,   0,   0],
         [351, 335, 773,  ...,   0,   0,   0]], device='cuda:0'),
 tensor([[  6,   3,  33,  ...,   0,   0,   0],
         [332,  17,  18,  ...,   0,   0,   0],
         [188, 186, 401,  ...,   0,   0,   0],
         ...,
         [924, 567, 557,  ...,   0,   0,   0],
         [253, 136,  99,  ...,  86, 220, 214],
         [  8,   9, 127,  ...,   0,   0,   0]], device='cuda:0'),
 tensor([[712, 744, 712,  ...,   0,   0,   0],
         [255, 256, 242,  ...,   0,   0,   0],
         [121, 169, 385,  ..., 419,  29, 242],
         ...,
         [457, 268, 269,  ...,   0,   0,   0],
         [167, 580, 365,  ...,   0,   0,   0],
         [ 42,  44, 270,  ...,   0,   0,   0]], device='cuda:0'))

In [ ]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextRNN(nn.Module):
    def __init__(self, parameter):
        super(TextRNN, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        word_size = parameter['word_size']
        self.embedding = nn.Embedding(word_size, embedding_dim, padding_idx=0)
        
        self.lstm_q = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)

        self.lstm_a = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)


        
    def forward(self, q, a1,a2 = None):
        q_emd = self.embedding(q)
        q_emd,(h, c)= self.lstm_q(q_emd)
        q_emd = torch.max(q_emd,1)[0]

        a1_emd = self.embedding(a1)
        a1_emd,(h, c)= self.lstm_a(a1_emd)
        a1_emd = torch.max(a1_emd,1)[0]
        if a2 is not None:
            a2_emd = self.embedding(a2)
            a2_emd,(h, c)= self.lstm_a(a2_emd)
            a2_emd = torch.max(a2_emd,1)[0]
            return q_emd,a1_emd,a2_emd
        return F.cosine_similarity(q_emd,a1_emd,1,1e-8)

In [ ]:
#若不生成parameter.pkl这里执行会报错
test_model = TextRNN(parameter).cuda()#我电脑不支持
#test_model = TextRNN(parameter)
test_model(test_q,test_a)

tensor([0.7633, 0.6893, 0.6920, 0.6954, 0.6982, 0.5583, 0.4576, 0.7430, 0.5634,
        0.7262, 0.6906, 0.7603, 0.6655, 0.6542, 0.7458, 0.5741, 0.6039, 0.7439,
        0.7588, 0.5774, 0.7130, 0.5730, 0.5830, 0.7816, 0.7627, 0.7156, 0.5825,
        0.5894, 0.7555, 0.6349, 0.5226, 0.5891, 0.5770, 0.5384, 0.6198, 0.6474,
        0.6709, 0.7471, 0.5490, 0.5584, 0.6946, 0.7428, 0.8014, 0.6349, 0.6196,
        0.5712, 0.5387, 0.6290, 0.7213, 0.5635, 0.7820, 0.7837, 0.5872, 0.5043,
        0.5929, 0.6259, 0.6113, 0.7342, 0.6185, 0.6778, 0.5483, 0.6972, 0.7904,
        0.7724, 0.7117, 0.7579, 0.7753, 0.5683, 0.6988, 0.7679, 0.5399, 0.6010,
        0.6570, 0.7285, 0.7914, 0.7123, 0.7047, 0.7027, 0.7394, 0.5302, 0.7845,
        0.7177, 0.7728, 0.8209, 0.7384, 0.6765, 0.7345, 0.5591, 0.7348, 0.7454,
        0.8218, 0.6979, 0.7628, 0.8118, 0.5357, 0.5569, 0.6010, 0.6858, 0.6730,
        0.5946, 0.4749, 0.7748, 0.4931, 0.7344, 0.7797, 0.5212, 0.7055, 0.8202,
        0.5419, 0.7504, 0.7763, 0.6707, 

In [ ]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 构建模型
model = TextRNN(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.1, momentum=0.95, nesterov=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)

# 准备迭代器
train_yield = batch_yield(parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')

if os.path.exists('grade.h5'):
  print("grade.h5 exist!")
else:
  print("grade.h5 not exist!")
  while 1:
          q,a,n,epoch,keys = next(train_yield)
          if keys:
              break
          q_emd,a_emd,n_emd = model(q,a,n)
          loss = nn.functional.triplet_margin_loss(q_emd, a_emd, n_emd,reduction='mean')
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss_cal.append(loss.item())
          if epoch is not None:
              if (epoch+1)%1 == 0:
                  loss_cal = sum(loss_cal)/len(loss_cal)
                  if loss_cal < min_loss:
                      min_loss = loss_cal
                      torch.save(model.state_dict(), 'grade.h5')
                  print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                        parameter['epoch'],loss_cal))
                  optimizer.step()
              loss_cal = [loss.item()]


grade.h5 not exist!



 22%|██▏       | 299/1390 [01:24<05:09,  3.53it/s]

100%|██████████| 1390/1390 [00:00<00:00, 3296.57it/s]


epoch [1/100], Loss: 0.9623


100%|██████████| 1390/1390 [00:00<00:00, 3588.27it/s]


epoch [2/100], Loss: 0.9286


...

epoch [98/100], Loss: 0.0294


100%|██████████| 1390/1390 [00:00<00:00, 3626.24it/s]


epoch [99/100], Loss: 0.0248


100%|██████████| 1390/1390 [00:00<00:00, 3594.71it/s]

epoch [100/100], Loss: 0.0277
